In [1]:
!pip install grpcio==1.42.0 tensorflow-serving-api==2.7.0

  Using cached grpcio-1.42.0.tar.gz (21.3 MB)
  Preparing metadata (setup.py) ... done
  Using cached tensorflow_serving_api-2.7.0-py2.py3-none-any.whl (37 kB)
  Using cached tensorboard-2.15.0-py3-none-any.whl (5.6 MB)
INFO: pip is looking at multiple versions of opt-einsum to determine which version is compatible with other requirements. This could take a while.
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
INFO: pip is looking at multiple versions of numpy to determine which version is compatible with other requirements. This could take a while.
  Using cached numpy-1.26.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
INFO: pip is looking at multiple versions of ml-dtypes to determine which version is compatible with other requirements. This could take a while.
  Using cached ml_dtypes-0.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
INFO: pip is looking at multiple versions of libclang to determine which version is compatib

In [2]:
!pip install keras-image-helper

In [3]:
import grpc
import tensorflow as tf
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc
from keras_image_helper import create_preprocessor

2023-12-19 13:55:52.231313: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-19 13:55:52.653769: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-19 13:55:52.654178: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-19 13:55:52.736504: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-19 13:55:52.953634: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-19 13:55:52.962683: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [4]:
host = "localhost:8500"
channel = grpc.insecure_channel(host)
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [5]:
preprocessor = create_preprocessor("resnet50", target_size=(224, 224))

In [6]:
path = "../illustrations/guess-me.jpg"
x = preprocessor.from_path(path)

In [7]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [8]:
pb_request = predict_pb2.PredictRequest()
pb_request.model_spec.name = "cats-classifier"
pb_request.model_spec.signature_name = "serving_default"

pb_request.inputs["input_6"].CopyFrom(np_to_protobuf(x))

In [9]:
pb_response = stub.Predict(pb_request, timeout=20.0)

In [10]:
pb_response

model_spec {
  name: "cats-classifier"
  version {
    value: 1
  }
  signature_name: "serving_default"
}
outputs {
  key: "dense_5"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 1
      }
      dim {
        size: 20
      }
    }
    float_val: 6.78086254e-09
    float_val: 6.06849471e-06
    float_val: 0.0137340333
    float_val: 7.3563956e-07
    float_val: 7.85281354e-11
    float_val: 4.44484662e-08
    float_val: 5.92079274e-10
    float_val: 0.000760414638
    float_val: 6.2603851e-09
    float_val: 0.0597831644
    float_val: 1.00058526e-06
    float_val: 3.46500087e-07
    float_val: 2.5057156e-07
    float_val: 4.04635975e-05
    float_val: 0.000111605688
    float_val: 6.33200647e-09
    float_val: 0.925525248
    float_val: 1.71383803e-07
    float_val: 4.12557974e-11
    float_val: 3.64106563e-05
  }
}

In [11]:
preds = pb_response.outputs["dense_5"].float_val

In [12]:
classes = [
    "Ragdoll",
    "Bombay",
    "American Curl",
    "Birman",
    "Egyptian Mau",
    "Bengal",
    "Siamese",
    "Maine Coon",
    "American Bobtail",
    "Turkish Angora",
    "Russian Blue",
    "British Shorthair",
    "Persian",
    "Abyssinian",
    "Norwegian Forest",
    "Manx",
    "Scottish Fold",
    "American Shorthair",
    "Exotic Shorthair",
    "Sphynx",
]

dict(zip(classes, preds))

{'Ragdoll': 6.780862538136034e-09,
 'Bombay': 6.068494712963002e-06,
 'American Curl': 0.013734033331274986,
 'Birman': 7.356395599344978e-07,
 'Egyptian Mau': 7.852813538322678e-11,
 'Bengal': 4.444846624096499e-08,
 'Siamese': 5.920792744973369e-10,
 'Maine Coon': 0.0007604146376252174,
 'American Bobtail': 6.260385099210453e-09,
 'Turkish Angora': 0.05978316441178322,
 'Russian Blue': 1.0005852573158336e-06,
 'British Shorthair': 3.4650008728931425e-07,
 'Persian': 2.505715599454561e-07,
 'Abyssinian': 4.0463597542839125e-05,
 'Norwegian Forest': 0.00011160568828927353,
 'Manx': 6.332006474707441e-09,
 'Scottish Fold': 0.9255252480506897,
 'American Shorthair': 1.7138380314918322e-07,
 'Exotic Shorthair': 4.1255797389450066e-11,
 'Sphynx': 3.64106563210953e-05}